In [21]:
import pymysql
import json
def read_userinfo(file_path, encodings=['utf-8']):
    userinfo_list = []
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                for line in file:
                    userinfo = line.strip().split(' ')
                    if userinfo[0] == '남성':
                        userGender = True
                    else:
                        userGender = False
                    userAge = int(userinfo[1])
                    userFavorite = userinfo[2].split(',')
                    userinfo_list.append([userAge, userGender, userFavorite])
            return userinfo_list
        except UnicodeDecodeError:
            continue  
        except FileNotFoundError:
            print(f"Error: The file {file_path} does not exist.")
            return []
    print(f"Error: Could not decode file {file_path} with given encodings.")
    return []

file_path = "init_user_info.txt"
user_infos = read_userinfo(file_path)

for userinfo in user_infos:
    print(userinfo)

try:
    conn = pymysql.connect(host='localhost', user='root', password='pangpar1', db='still88', charset='utf8')
    cursor = conn.cursor()

    for i, userinfo in enumerate(user_infos):
        user_nickname = "USER_" + str(i+1)
        user_favorite = []
        for category in userinfo[2]:
            cursor.execute(f"SELECT recipeId from recipe where recipeCategory = \"{category}\";")
            recipeIds = cursor.fetchall()
            for recipeId in recipeIds:
                user_favorite.append(recipeId[0])   
        user_favorite.sort()
        query = "INSERT INTO User(user_nickName, user_age, user_gender, user_favorite) VALUES (%s, %s, %s, %s);"
        cursor.execute(query, (user_nickname, userinfo[0], userinfo[1], json.dumps(user_favorite)))

    conn.commit()
except pymysql.MySQLError as e:
    print(f"Error: {e}")
finally:
    cursor.close()
    conn.close()


[23, True, ['한식', '양식', '고기']]
[22, True, ['한식', '일식', '중식', '양식', '아시안', '찜탕', '고기', '죽', '채소']]
[21, True, ['한식', '일식', '중식', '양식', '아시안', '찜탕', '고기', '죽', '채소']]
[23, False, ['한식', '일식', '양식', '고기', '채소']]
[23, False, ['한식', '일식', '양식', '아시안', '고기']]
[21, True, ['한식', '중식', '양식']]
[19, False, ['일식', '양식', '아시안', '고기']]
[19, True, ['한식', '일식', '중식', '양식', '아시안', '찜탕', '고기']]
[19, True, ['한식', '일식', '고기']]
[22, True, ['한식', '일식', '중식', '양식', '고기']]
[18, True, ['일식', '고기']]
[21, False, ['한식', '일식', '중식', '양식', '아시안', '고기', '죽', '채소']]
[22, False, ['한식', '일식', '중식', '양식', '아시안', '찜탕', '고기', '죽', '채소']]
[24, False, ['한식', '일식', '양식', '찜탕', '고기']]
[26, True, ['일식', '중식', '양식', '고기']]
[21, True, ['한식', '일식', '고기']]
[23, False, ['한식', '아시안']]
[48, False, ['한식', '중식', '양식', '찜탕', '고기', '채소']]
[21, False, ['한식', '일식', '중식', '양식', '아시안', '찜탕', '고기', '죽']]
[51, True, ['한식', '일식', '중식', '양식', '찜탕', '고기']]
[21, False, ['한식', '일식', '양식']]
[53, False, ['한식', '양식', '찜탕', '고기', '채소']]
[22, True, ['일식

In [22]:
import requests
import json
import pymysql

def request_data():
    apikey = '360bfc50b4154894b83c'
    request_url = f'https://openapi.foodsafetykorea.go.kr/api/{apikey}/COOKRCP01/json/1/1000'
    response = requests.get(request_url)
    data = response.json()

    recipes = []

    for item in data['COOKRCP01']['row']:
        recipe_name = item['RCP_NM']
        recipe = {'recipe_name': recipe_name, 'cook_steps': []}

        for i in range(1, 21):
            cook_step_key = f'MANUAL{i:02d}'
            cook_step = item.get(cook_step_key)
            if not cook_step or cook_step.strip() == '':
                break
            recipe['cook_steps'].append(cook_step.replace('\n', ' '))

        recipes.append(recipe)

    return recipes

def read_ingredients(file_path, encodings=['utf-8']):
    ingredients = []
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                for line in file:
                    ingredients.append(line.strip())
            return ingredients
        except UnicodeDecodeError:
            continue  
        except FileNotFoundError:
            print(f"Error: The file {file_path} does not exist.")
            return []
    print(f"Error: Could not decode file {file_path} with given encodings.")
    return []

def save_to_db(recipes, recipe_name, recipe_category, recipe_ingredients):
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='pangpar1',
        db='still88',
        charset='utf8'
    )
    cursor = connection.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS recipe (
            recipe_id INT AUTO_INCREMENT PRIMARY KEY,
            recipe_name VARCHAR(255),
            recipe_category VARCHAR(255),
            recipe_ingredient JSON,
            recipe_description JSON
        )
    ''')

    for i in range(len(recipe_name)):
        recipe_ingredient = recipe_ingredients[i].split(' ')[::2]
        ingredient_json = json.dumps(recipe_ingredient)

        index = i
        if index >= 493:
            index += 2

        if index >= 952:
            index += 1
            
        if index < len(recipes):
            recipe_description = json.dumps(recipes[index]['cook_steps'], ensure_ascii=False)
        else:
            recipe_description = json.dumps([])

        query = '''
            INSERT INTO recipe (recipe_name, recipe_category, recipe_ingredient, recipe_description)
            VALUES (%s, %s, %s, %s)
        '''
        cursor.execute(query, (recipe_name[i], recipe_category[i], ingredient_json, recipe_description))

    connection.commit()
    cursor.close()
    connection.close()

def main():
    recipes = request_data()

    file_path1 = "recipe_name.txt"
    recipe_name = read_ingredients(file_path1)

    file_path2 = "recipe_category.txt"
    recipe_category = read_ingredients(file_path2)

    file_path3 = "recipe_ingredients.txt"
    recipe_ingredients = read_ingredients(file_path3)

    if len(recipe_name) != len(recipe_category) or len(recipe_name) != len(recipe_ingredients):
        print("Error: The number of recipe names, categories, and ingredients do not match.")
    else:
        save_to_db(recipes, recipe_name, recipe_category, recipe_ingredients)

if __name__ == "__main__":
    main()


OperationalError: (1054, "Unknown column 'recipe_name' in 'field list'")

In [ ]:
import pymysql

def read_ingredients(file_path, encodings=['utf-8']):
    ingredients = []
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                for line in file:
                    ingredients.append(line.strip())
            return ingredients
        except UnicodeDecodeError:
            continue  
        except FileNotFoundError:
            print(f"Error: The file {file_path} does not exist.")
            return []
    print(f"Error: Could not decode file {file_path} with given encodings.")
    return []

file_path = "ingredient_name.txt"
ingredient_name = read_ingredients(file_path)

file_path2 = "ingredient_category.txt"
ingredient_category = read_ingredients(file_path2)

if len(ingredient_name) != len(ingredient_category):
    print("Error: The number of ingredients does not match the number of categories.")
    print(len(ingredient_category))
    print(len(ingredient_name))
else:
    try:
        # mysql에 대한 정보 변경하세요
        conn = pymysql.connect(host='localhost', user='root', password='pangpar1', db='still88', charset='utf8')
        cursor = conn.cursor()

        for i in range(len(ingredient_name)):
            query = "INSERT INTO ingredient(ingredient_category, ingredient_name) VALUES (%s, %s);"
            cursor.execute(query, (ingredient_category[i], ingredient_name[i]))

        conn.commit()
    except pymysql.MySQLError as e:
        print(f"Error: {e}")
    finally:
        cursor.close()
        conn.close()
